In [1]:
import numpy as np
import sympy as sp

In [2]:
class Node:
    def __init__(self, operation=None, left=None, right=None, value=None):
        self.operation = operation
        self.left = left
        self.right = right
        self.value = value
        
    def evaluate(self):
        if self.operation is None:
            return self.value
        elif self.operation == '+':
            return self.left.evaluate() + self.right.evaluate()
        elif self.operation == '*':
            return self.left.evaluate() * self.right.evaluate()
        
        
    def __repr__(self):
        """노드 정보 보기 쉽게 출력"""
        if self.operation is None:
            return f"({self.value})"
        return f"({self.left} {self.operation} {self.right})"
        

In [3]:
input_nodes = [
    Node(value=3),
    Node(value=2),
    Node(value=1),
    Node(value=7),
    Node(value=5),
    Node(value=4),
]

In [4]:
gate1 = Node(operation='*', left=input_nodes[0], right=input_nodes[1])
gate2 = Node(operation='*', left=gate1, right=Node(operation='+', left=input_nodes[2], right=input_nodes[3]))
gate3 = Node(operation='*', left=Node(operation='+', left=input_nodes[2], right=input_nodes[3]),  right=Node(operation='+', left=input_nodes[4], right=input_nodes[5]))

In [5]:
print("Gate Structure:")
print(f"Gate 1: {gate1}")
print(f"Gate 2: {gate2}")
print(f"Gate 3: {gate3}")

print("\nEvaluations:")
print(f"Gate 1 Result: {gate1.evaluate()}")
print(f"Gate 2 Result: {gate2.evaluate()}")
print(f"Gate 3 Result: {gate3.evaluate()}")

Gate Structure:
Gate 1: ((3) * (2))
Gate 2: (((3) * (2)) * ((1) + (7)))
Gate 3: (((1) + (7)) * ((5) + (4)))

Evaluations:
Gate 1 Result: 6
Gate 2 Result: 48
Gate 3 Result: 72


In [6]:
transcript_table = [3, 2, 1, 7, 5, 4, 6, 48, 72]
gate_list = [gate1, gate2, gate3]


def find_left_selector(gate, gate_idx, c, selector):    
    if gate.operation == None:
        if gate.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")    
    elif gate.left.operation == None:
        if gate.left.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")
    elif gate.left.operation == '+':
        find_left_selector(gate.left.left, gate_idx, c, selector)
        find_left_selector(gate.left.right, gate_idx, c, selector)
    elif gate.operation == '*':
        if gate.left.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")

    return selector

def find_right_selector(gate, gate_idx, c, selector):    
    if gate.operation == None:
        if gate.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")    
    elif gate.right.operation == None:
        if gate.right.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")
    elif gate.right.operation == '+':
        find_right_selector(gate.right.right, gate_idx, c, selector)
        find_right_selector(gate.right.left, gate_idx, c, selector)
    elif gate.operation == '*':
        if gate.right.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")

    return selector

def gate_selector(gate_list, direction, transcript_table):
    polynomial_table = []
    if direction == 'left':
        for c in transcript_table:
            selector = [0] * len(gate_list)
            for gate_idx, gate in enumerate(gate_list):
                find_left_selector(gate, gate_idx, c, selector)
            polynomial_table.append(selector)
            
    elif direction == 'right':
        for c in transcript_table:
            selector = [0] * len(gate_list)
            for gate_idx, gate in enumerate(gate_list):
                find_right_selector(gate, gate_idx, c, selector)
            polynomial_table.append(selector)
            
    return polynomial_table


In [7]:
gate_selector(gate_list, 'left', transcript_table)

gate_idx: 0, c: 3, selector: [1, 0, 0]
gate_idx: 2, c: 1, selector: [0, 0, 1]
gate_idx: 2, c: 7, selector: [0, 0, 1]
gate_idx: 1, c: 6, selector: [0, 1, 0]


[[1, 0, 0],
 [0, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [0, 1, 0],
 [0, 0, 0],
 [0, 0, 0]]

In [8]:
gate_selector(gate_list, 'right', transcript_table)

gate_idx: 0, c: 2, selector: [1, 0, 0]
gate_idx: 1, c: 1, selector: [0, 1, 0]
gate_idx: 1, c: 7, selector: [0, 1, 0]
gate_idx: 2, c: 5, selector: [0, 0, 1]
gate_idx: 2, c: 4, selector: [0, 0, 1]


[[0, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0]]

In [9]:
def find_selector(gate, gate_idx, c, selector, direction):
    """Left 또는 Right Selector Polynomial을 찾는 함수"""
    if gate is None:
        return
    
    # 리프 노드라면 값을 확인
    if gate.operation is None:
        if gate.evaluate() == c:
            selector[gate_idx] = 1        
        return
    
    # 왼쪽 / 오른쪽 / 출력 선택
    if direction == 'left':
        target = gate.left 
    elif direction == 'right':
        target = gate.right
    elif direction == 'output':
        target = gate

    # 리프 값이 맞는 경우
    if target and target.operation is None and target.evaluate() == c:
        selector[gate_idx] = 1
        return

    # 덧셈 노드인 경우 재귀 호출
    if target and target.operation == '+':
        find_selector(target.left, gate_idx, c, selector, direction)
        find_selector(target.right, gate_idx, c, selector, direction)

    # 곱셈 노드이면서 직접 값이 맞는 경우
    if gate.operation == '*' and target.evaluate() == c:
        selector[gate_idx] = 1

def gate_selector(gate_list, direction, transcript_table):
    """게이트에서 left 또는 right selector polynomial을 찾는 함수"""
    polynomial_table = []
    for c in transcript_table:
        selector = [0] * len(gate_list)
        for gate_idx, gate in enumerate(gate_list):
            find_selector(gate, gate_idx, c, selector, direction)
        polynomial_table.append(selector)
    return polynomial_table



In [10]:

# 실행 예시 (left selector 생성)
left_selector_table = gate_selector(gate_list, 'left', transcript_table)
print("Left Selector Table:")
left_selector_table

Left Selector Table:


[[1, 0, 0],
 [0, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [0, 1, 0],
 [0, 0, 0],
 [0, 0, 0]]

In [11]:

# 실행 예시 (left selector 생성)
right_selector_table = gate_selector(gate_list, 'right', transcript_table)
print("Right Selector Table:")
right_selector_table

Right Selector Table:


[[0, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0]]

In [12]:

# 실행 예시 (left selector 생성)
output_selector_table = gate_selector(gate_list, 'output', transcript_table)
print("output Selector Table:")
output_selector_table

output Selector Table:


[[0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 0, 1]]

In [13]:
def properties(transcript_table, gate_selector, omega):
    result = 0
    for idx, c in enumerate(transcript_table):
        result += c * gate_selector[idx][omega]
        
    return result
    

In [14]:
print(properties(transcript_table, left_selector_table, 0))
print(properties(transcript_table, left_selector_table, 1))
print(properties(transcript_table, left_selector_table, 2))

3
6
8


In [15]:
print(properties(transcript_table, right_selector_table, 0))
print(properties(transcript_table, right_selector_table, 1))
print(properties(transcript_table, right_selector_table, 2))

2
8
9


In [16]:
print(properties(transcript_table, output_selector_table, 0))
print(properties(transcript_table, output_selector_table, 1))
print(properties(transcript_table, output_selector_table, 2))

6
48
72


In [17]:
def master_polynomial(transcript_table, left_selector_table, right_selector_table, output_selector_table, omega):
    left_properties = properties(transcript_table, left_selector_table, omega)
    right_properties = properties(transcript_table, right_selector_table, omega)
    output_properties = properties(transcript_table, output_selector_table, omega)
    
    return left_properties * right_properties - output_properties

In [18]:
print(master_polynomial(transcript_table, left_selector_table, right_selector_table, output_selector_table, 0))
print(master_polynomial(transcript_table, left_selector_table, right_selector_table, output_selector_table, 1))
print(master_polynomial(transcript_table, left_selector_table, right_selector_table, output_selector_table, 2))

0
0
0


In [19]:
def properties(transcript_table, gate_selector):
    """주어진 selector 테이블에 대해 property 값을 계산하는 함수"""
    def compute(omega):
        return sum(c * gate_selector[idx][omega] for idx, c in enumerate(transcript_table))
    return compute

def master_polynomial(transcript_table, selector_tables):
    """QAP의 master polynomial을 계산하는 클로저"""
    def compute(omega):
        left_properties = properties(transcript_table, selector_tables['left'])(omega)
        right_properties = properties(transcript_table, selector_tables['right'])(omega)
        output_properties = properties(transcript_table, selector_tables['output'])(omega)

        return left_properties * right_properties - output_properties

    return compute


In [20]:
selector_tables = {
    'left': left_selector_table,
    'right': right_selector_table,
    'output': output_selector_table
}

master_poly = master_polynomial(transcript_table, selector_tables)

# 특정한 omega 값으로 평가
print(master_poly(0))
print(master_poly(1))
print(master_poly(2))


0
0
0


In [21]:
def vanishing_polynomial(omegas):
    """주어진 selector 테이블에 대해 vanishing polynomial을 계산하는 함수"""
    def compute(x):
        return (x - omegas[0]) * (x - omegas[1]) * (x - omegas[2])
    
    return compute

In [22]:
class FiniteField:
    def __init__(self, value, prime):
        if prime <= 1:
            raise ValueError("Prime must be greater than 1")
        self.value = value % prime
        self.prime = prime
        
    def to_extension(self, r):
        """F_p 원소를 F_p^2로 변환"""
        return FiniteFieldExtension(self.value, 0, self.prime, r)
        
    def __add__(self, other):
        if not isinstance(other, FiniteField):
            raise TypeError("Operand must be of type FiniteField")
        if self.prime != other.prime:
            raise ValueError("Primes must be the same")
        return FiniteField(self.value + other.value, self.prime)
    
    def __sub__(self, other):
        if not isinstance(other, FiniteField):
            raise TypeError("Operand must be of type FiniteField")
        if self.prime != other.prime:
            raise ValueError("Primes must be the same")
        return FiniteField(self.value - other.value, self.prime)

    def __mul__(self, other):
        if not isinstance(other, FiniteField):
            raise TypeError("Operand must be of type FiniteField")
        if self.prime != other.prime:
            raise ValueError("Primes must be the same")
        return FiniteField(self.value * other.value, self.prime)
    
    def __neg__(self):
        return FiniteField(-self.value, self.prime)

    def __truediv__(self, other):
        if not isinstance(other, FiniteField):
            raise TypeError("Operand must be of type FiniteField")
        if self.prime != other.prime:
            raise ValueError("Primes must be the same")
        if other.value == 0:
            raise ZeroDivisionError("Cannot divide by zero in Finite Field")
        
        return FiniteField(self.value * pow(other.value, -1, self.prime), self.prime)
    
    def __eq__(self, other):
        return isinstance(other, FiniteField) and self.value == other.value and self.prime == other.prime
    
    def __hash__(self):
        """ Make FiniteField hashable so it can be used as a dictionary key """
        return hash((self.value, self.prime))

    def __repr__(self):
        return f"FiniteField({self.value}, {self.prime})"



In [23]:
class FiniteFieldExtension:
    def __init__(self, b, c, prime, r):
        """F_p^2 = F_p[alpha], alpha^2 = r"""
        self.b = FiniteField(b, prime)
        self.c = FiniteField(c, prime)
        self.prime = prime
        self.r = FiniteField(r, prime)
        
        squares = set((i * i) % prime for i in range(prime))
        if self.r.value in squares:
            raise ValueError(f"{r} is a square in F_{prime}, cannot use for extension")
        
    def __add__(self, other:"FiniteFieldExtension") -> "FiniteFieldExtension":
        if self.prime != other.prime:
            raise ValueError("Primes must be the same")
        
        return FiniteFieldExtension(
            (self.b + other.b).value,
            (self.c + other.c).value,
            self.prime,
            self.r.value
        )
        
    def __sub__(self, other):
        if self.prime != other.prime or self.r != other.r:
            raise ValueError("Fields must be the same")
        return FiniteFieldExtension(
            (self.b - other.b).value,
            (self.c - other.c).value,
            self.prime,
            self.r.value
        )
        
    def __mul__(self, other):
        if self.prime != other.prime or self.r != other.r:
            raise ValueError("Fields must be the same")
        b_new = (self.b * other.b + self.c * other.c * self.r).value
        c_new = (self.b * other.c + self.c * other.b).value
        return FiniteFieldExtension(b_new, c_new, self.prime, self.r.value)
    
    def __truediv__(self, other):
        if self.prime != other.prime or self.r != other.r:
            raise ValueError("Fields must be the same")
        if other.b.value == 0 and other.c.value == 0:
            raise ZeroDivisionError("Cannot divide by zero in FiniteFieldExtension")
        denominator = (other.b * other.b - other.c * other.c * self.r).value
        if denominator == 0:
            raise ZeroDivisionError("Denominator is zero in FiniteFieldExtension")
        b_num = (self.b * other.b + self.c * other.c * self.r).value
        c_num = (self.c * other.b - self.b * other.c).value
        denom_inv = pow(denominator, -1, self.prime)
        b_new = (b_num * denom_inv) % self.prime
        c_new = (c_num * denom_inv) % self.prime
        return FiniteFieldExtension(b_new, c_new, self.prime, self.r.value)

    def __neg__(self):
        return FiniteFieldExtension((-self.b).value, (-self.c).value, self.prime, self.r.value)

    def __eq__(self, other):
        return (isinstance(other, FiniteFieldExtension) and
                self.b == other.b and self.c == other.c and
                self.prime == other.prime and self.r == other.r)

    def __hash__(self):
        return hash((self.b.value, self.c.value, self.prime, self.r.value))

    def __repr__(self):
        return f"FiniteFieldExtension({self.b.value} + {self.c.value}*alpha, prime={self.prime}, alpha^2={self.r.value})"
    
    def inverse(self):
        if self.b.value == 0 and self.c.value == 0:
            raise ZeroDivisionError("Cannot invert zero in FiniteFieldExtension")
        denominator = (self.b * self.b - self.c * self.c * self.r).value
        if denominator == 0:
            raise ZeroDivisionError("Denominator is zero in FiniteFieldExtension")
        denom_inv = pow(denominator, -1, self.prime)
        b_new = (self.b.value * denom_inv) % self.prime
        c_new = ((-self.c.value) * denom_inv) % self.prime
        return FiniteFieldExtension(b_new, c_new, self.prime, self.r.value)
        
        

In [24]:
# F_5^2에서의 점 정의
x = FiniteFieldExtension(1, 0, 5, 2)  # 1
y = FiniteFieldExtension(0, 1, 5, 2)  # alpha
point_Q = (x, y)  # (1, alpha)

# F_5에서의 점을 F_5^2로 확장
x_p = FiniteFieldExtension(0, 0, 5, 2)  # 0
y_p = FiniteFieldExtension(1, 0, 5, 2)  # 1
point_P = (x_p, y_p)  # (0, 1)

In [25]:
print(point_P)
print(point_Q)

(FiniteFieldExtension(0 + 0*alpha, prime=5, alpha^2=2), FiniteFieldExtension(1 + 0*alpha, prime=5, alpha^2=2))
(FiniteFieldExtension(1 + 0*alpha, prime=5, alpha^2=2), FiniteFieldExtension(0 + 1*alpha, prime=5, alpha^2=2))


In [26]:
class EllipticCurve:
    def __init__(self, a, b, prime):
        '''y^2 = x^3 + ax + b mod prime'''
        self.a = a
        self.b = b
        self.prime = prime
        self.infinity = None
        self.points = [self.infinity]

        if isinstance(a, FiniteField) and isinstance(b, FiniteField):
            self.field_type = "FiniteField"
            self.r = None
            self.discriminant = (FiniteField(4, self.prime) * (self.a * self.a * self.a) + 
                                FiniteField(27, self.prime) * (self.b * self.b)).value
            if self.discriminant % self.prime == 0:
                raise ValueError("This curve is not elliptic (discriminant is zero)")

            for x_val in range(self.prime):
                for y_val in range(self.prime):
                    x = FiniteField(x_val, self.prime)
                    y = FiniteField(y_val, self.prime)
                    if self.is_point_on_curve(x, y):
                        self.points.append((x, y))
                        
        elif isinstance(a, FiniteFieldExtension) and isinstance(b, FiniteFieldExtension):
            self.field_type = "FiniteFieldExtension"
            self.r = a.r.value
            self.discriminant = (FiniteFieldExtension(4, 0, self.prime, self.r) * (self.a * self.a * self.a) + 
                                FiniteFieldExtension(27, 0, self.prime, self.r) * (self.b * self.b))
            if self.discriminant.b.value == 0 and self.discriminant.c.value == 0:
                raise ValueError("This curve is not elliptic (discriminant is zero)")

            squares = set()
            for b_val_y in range(self.prime):
                for c_val_y in range(self.prime):
                    y = FiniteFieldExtension(b_val_y, c_val_y, self.prime, self.r)
                    y2 = y * y
                    squares.add((y2.b.value, y2.c.value))

            for b_val in range(self.prime):
                for c_val in range(self.prime):
                    x = FiniteFieldExtension(b_val, c_val, self.prime, self.r)
                    rhs = x * x * x + self.a * x + self.b
                    if (rhs.b.value, rhs.c.value) in squares:
                        for b_val_y in range(self.prime):
                            for c_val_y in range(self.prime):
                                y = FiniteFieldExtension(b_val_y, c_val_y, self.prime, self.r)
                                lhs = y * y
                                if lhs.b.value == rhs.b.value and lhs.c.value == rhs.c.value:
                                    self.points.append((x, y))
        else:
            raise ValueError("a and b must both be either FiniteField or FiniteFieldExtension instances")

    def is_point_on_curve(self, x, y):
        left = (y * y)
        right = (x * x * x + self.a * x + self.b)
        if self.field_type == "FiniteField":
            return left.value == right.value % self.prime
        else:
            return left.b.value == right.b.value and left.c.value == right.c.value

    def add_points(self, p1, p2):
        if p1 not in self.points or p2 not in self.points:
            raise ValueError(f"Points are not on the curve: p1={p1}, p2={p2}")
        
        if p1 is self.infinity:
            return p2
        if p2 is self.infinity:
            return p1     

        x1, y1 = p1
        x2, y2 = p2
        
        if x1 == x2 and y1 != y2:
            return None
        
        if p1 == p2:
            if (isinstance(y1, FiniteField) and y1.value == 0) or \
               (isinstance(y1, FiniteFieldExtension) and y1.b.value == 0 and y1.c.value == 0):
                return None
            if self.field_type == "FiniteField":
                slope = (FiniteField(3, self.prime) * x1 * x1 + self.a) / (FiniteField(2, self.prime) * y1)
            else:
                slope = (FiniteFieldExtension(3, 0, self.prime, self.r) * x1 * x1 + self.a) / \
                        (FiniteFieldExtension(2, 0, self.prime, self.r) * y1)
        else:
            if x1 == x2:
                return None
            slope = (y2 - y1) / (x2 - x1)
            
        x3 = slope * slope - x1 - x2
        y3 = slope * (x1 - x3) - y1

        # 중복 제거를 하지 않으므로, 결과가 curve.points에 없어도 추가
        result = (x3, y3)
        if result not in self.points:
            self.points.append(result)
        return result
    
    def multiply_point(self, p, n):
        if p is None:
            return None
        result = None
        temp = p
        while n > 0:
            if n % 2 == 1:
                result = self.add_points(result, temp)
            temp = self.add_points(temp, temp)
            n //= 2
        return result
    
    def find_points_of_order_n(self, n):
        result = []
        for point in self.points:
            if point is None:
                continue
            if self.multiply_point(point, n) == self.infinity:
                order = 1
                current = point
                while current != self.infinity and order <= n:
                    current = self.add_points(current, point)
                    order += 1
                if order == n:
                    result.append(point)
        return result
    
    def is_linear_independent(self, p1, p2, n=6):
        if p1 == self.infinity or p2 == self.infinity:
            return False
        
        # p1을 F_p^2로 확장
        x1, y1 = p1
        if isinstance(x1, FiniteField) and isinstance(y1, FiniteField):
            extended_p1 = (x1.to_extension(self.r), y1.to_extension(self.r))
        else:
            extended_p1 = p1
        
        # a*p1 + b*p2 = O 확인
        for a in range(n):
            for b in range(n):
                if a == 0 and b == 0:
                    continue
                if self.add_points(self.multiply_point(extended_p1, a), self.multiply_point(p2, b)) == self.infinity:
                    return False
        return True
    
    def __repr__(self):
        return f"EllipticCurve(y^2 = x^3 + {self.a}x + {self.b} over F_{self.prime})"
		

In [27]:
from collections import Counter

class Divisor:
    def __init__(self, curve: EllipticCurve):
        self.curve = curve
        self.points = curve.points
        self.prime = curve.prime
        self.r = self.points[1][0].r.value
        
    def generate_equiv_divisors(self, point, point_R):
        """(P) - (O)와 동등한 divisor 생성: (P + R) - (R)"""
        P_plus_R = self.curve.add_points(point, point_R)
        divisor = Counter()
        divisor[P_plus_R] = 1
        divisor[point_R] = -1
        return divisor
    
    def compute_divisor_of_function(self, function, is_vertical=False, is_tangent=False):
        divisor = Counter()
        a, b, c = function
        number_of_zeros = 0

        # 영점 계산
        for point in self.points:
            if point == self.curve.infinity:
                continue
            x, y = point
            value = a * x + b * y + c
            if value.b.value == 0 and value.c.value == 0:  # FiniteFieldExtension에서 0 판단
                divisor[point] += 1  # 영점 계수 증가
                number_of_zeros += 1

        # 극점 차수 설정
        if is_vertical:
            # 수직선의 경우 극점 차수는 -2 (타원 곡선과 교점이 2개)
            divisor[self.curve.infinity] = -2
        elif is_tangent:
            # 접선의 경우 극점 차수는 -3 (중복 교점 포함)
            divisor[self.curve.infinity] = -3
        else:
            # 일반 직선의 경우 극점 차수는 -3 (타원 곡선과 교점이 3개)
            divisor[self.curve.infinity] = -3

        return divisor
            
    def evaluate_function(self, divisor, linear_function, is_vertical=False, is_tangent=False):
        # 직선 함수의 divisor 계산
        #function_divisor = self.compute_divisor_of_function(linear_function, is_vertical, is_tangent)
        result = FiniteFieldExtension(1, 0, self.curve.prime, self.r)
        a, b, c = linear_function
        
        # divisor에 따라 값 평가
        for point, exponent in divisor.items():
            if point == self.curve.infinity:
                continue
            x, y = point
            value = a * x + b * y + c
            #print(f"Evaluating at {point}: {value} (function divisor at {point} = {function_divisor[point]})")
            if value.b.value == 0 and value.c.value == 0:
                raise ValueError("Function evaluation resulted in zero, which may cause invalid pairing.")
            if exponent > 0:
                for _ in range(exponent):
                    result = result * value
            elif exponent < 0:
                value_inv = value.inverse()
                for _ in range(-exponent):
                    result = result * value_inv
        
        # 극점 처리 (function_divisor의 극점 차수 반영)
        ''' 
            if function_divisor[self.curve.infinity] < 0:
                value_at_infinity = FiniteFieldExtension(1, 0, self.curve.prime, self.r)  # 극점에서의 값은 1로 가정
                for _ in range(-function_divisor[self.curve.infinity]):
                    result = result / value_at_infinity
            '''
        return result

In [28]:
class WeilPairing:
    def __init__(self, curve: EllipticCurve, P, Q):
            self.curve = curve
            self.P = P
            self.Q = Q
            self.r = self.curve.points[1][0].r.value
            if P not in curve.points or Q not in curve.points:
                raise ValueError("Points are not on the curve")
            self.div = Divisor(curve)
            
    def line_through_vertical(self, P):
        # 먼저 무한대점인지 확인
        if P == self.curve.infinity:
            return (FiniteFieldExtension(1, 0, self.curve.prime, self.r),
                    FiniteFieldExtension(0, 0, self.curve.prime, self.r),
                    FiniteFieldExtension(1, 0, self.curve.prime, self.r))
        # 그 다음에 언팩
        x, y = P
        return (FiniteFieldExtension(1, 0, self.curve.prime, self.r),
                FiniteFieldExtension(0, 0, self.curve.prime, self.r),
                -x)
    
    def line_through_points(self, P, Q):
        # 먼저 무한대점인지 확인
        if P == self.curve.infinity or Q == self.curve.infinity:
            return (FiniteFieldExtension(1, 0, self.curve.prime, self.r),
                    FiniteFieldExtension(0, 0, self.curve.prime, self.r),
                    FiniteFieldExtension(1, 0, self.curve.prime, self.r))
        
        # 그 다음에 언팩
        x1, y1 = P
        x2, y2 = Q
        
        # 나머지 로직은 동일
        if P == Q:
            if (isinstance(y1, FiniteField) and y1.value == 0) or \
            (isinstance(y1, FiniteFieldExtension) and y1.b.value == 0 and y1.c.value == 0):
                return (FiniteFieldExtension(1, 0, self.curve.prime, self.r),
                        FiniteFieldExtension(0, 0, self.curve.prime, self.r),
                        -x1)
            slope = (FiniteFieldExtension(3, 0, self.curve.prime, self.r) * x1 * x1 + self.curve.a) / \
                    (FiniteFieldExtension(2, 0, self.curve.prime, self.r) * y1)
            a = -slope
            b = FiniteFieldExtension(1, 0, self.curve.prime, self.r)
            c = slope * x1 - y1
            return (a, b, c)
        else:
            if x1 == x2:
                return (FiniteFieldExtension(1, 0, self.curve.prime, self.r),
                        FiniteFieldExtension(0, 0, self.curve.prime, self.r),
                        -x1)
            slope = (y2 - y1) / (x2 - x1)
            a = -slope
            b = FiniteFieldExtension(1, 0, self.curve.prime, self.r)
            c = y1 - slope * x1
            return (a, b, c)
                
    def miller(self, order, point, divisor):
        bin_order = bin(order)[2:]
        Z = point
        V = FiniteFieldExtension(1, 0, self.curve.prime, self.curve.r)
        
        for i in range(len(bin_order)):
            #print(f"Step {i}, Before Double: Z = {Z}, V = {V}")
            numerator = self.div.evaluate_function(divisor, self.line_through_points(Z, Z))
            denominator = self.div.evaluate_function(divisor, self.line_through_vertical(self.curve.add_points(Z, Z)))
            #print(f"numerator = {numerator}, denominator = {denominator}")
            V = V * V * (numerator / denominator)
            #print(f"After Double: V = {V}")
            Z = self.curve.add_points(Z, Z)
            if bin_order[i] == '1':
                #print(f"Step {i}, Before Add: Z = {Z}, V = {V}")
                Z = self.curve.add_points(Z, point)
                numerator = self.div.evaluate_function(divisor, self.line_through_points(Z, point))
                denominator = self.div.evaluate_function(divisor, self.line_through_vertical(Z))
                #print(f"numerator = {numerator}, denominator = {denominator}")
                V = V * (numerator / denominator)
                #print(f"After Add: V = {V}")
        return V

    def compute_pairing(self, order):
        if self.P == self.Q:
            return FiniteFieldExtension(1, 0, self.curve.prime, self.curve.r)
        
        max_attempts = 50
        for _ in range(max_attempts):
            try:
                R_P = random.choice([pt for pt in self.curve.points if pt != self.P and pt != self.curve.infinity])
                R_Q = random.choice([pt for pt in self.curve.points if pt != self.Q and pt != self.curve.infinity])

                divisor_P = self.div.generate_equiv_divisors(self.P, R_P)
                divisor_Q = self.div.generate_equiv_divisors(self.Q, R_Q)
                f_P_A_Q = self.miller(order, self.P, divisor_Q)
                f_Q_A_P = self.miller(order, self.Q, divisor_P)
                result = f_P_A_Q / f_Q_A_P
                # 추가 검증: 결과가 0이 아닌지 확인
                if result.b.value != 0 or result.c.value != 0:
                    # order가 n인지 확인
                    power_n = result
                    for i in range(order - 1):
                        power_n = power_n * result
                    if power_n == FiniteFieldExtension(1, 0, self.curve.prime, self.curve.r):
                        return result
            except ValueError:
                continue
        raise ValueError("Failed to compute pairing after multiple attempts")


In [30]:
import random
curve_p = EllipticCurve(a=FiniteField(0, 5), b=FiniteField(1, 5), prime=5)
curve_q = EllipticCurve(a=FiniteFieldExtension(0, 0, 5, 2), b=FiniteFieldExtension(1, 0, 5, 2), prime=5)

points_p_of_order_6 = [p for p in curve_p.find_points_of_order_n(6) if p is not None]
points_q_of_order_6 = [p for p in curve_q.find_points_of_order_n(6) if p is not None]

if not points_p_of_order_6:
    raise ValueError("No point of order 6 found in E(F_p). Change order n")
if not points_q_of_order_6:
    raise ValueError("No point of order 6 found in E(F_p^2). Change order n")

point_P = random.choice(points_p_of_order_6)
extended_point_P = (point_P[0].to_extension(2), point_P[1].to_extension(2))
point_Q = random.choice(points_q_of_order_6)
    
    
def test_properties_nth_root(curve_q, point_P, point_Q):
    print("Testing properties of Weil Pairing...")
    print(f"Curve: {curve_q}")
    print(f"Point P: {point_P}")
    print(f"Point Q: {point_Q}")
    # 선형 독립성 확인
    if not curve_q.is_linear_independent(point_P, point_Q):
        print("Points are linearly dependent, choosing new Q...")
        for q in points_q_of_order_6:
            if curve_q.is_linear_independent(point_P, q):
                point_Q = q
                break
        else:
            raise ValueError("Could not find linearly independent Q")

    weil_pairing = WeilPairing(curve_q, extended_point_P, point_Q)
    result = weil_pairing.compute_pairing(6)
    print(f'{1}th order: {result}')
    power_6 = result
    print("----------------")
    for i in range(1, 6):
        power_6 = power_6 * result
        print(f'{i+1}th order: {power_6}')        
    print("----------------")    
    assert power_6 == FiniteFieldExtension(1, 0, 5, 2), f"Identity property failed: e(P, Q)^6 = {power_6} != 1"
    print("Identity property test passed!")

In [31]:
test_properties_nth_root(curve_q, point_P, point_Q)

Testing properties of Weil Pairing...
Curve: EllipticCurve(y^2 = x^3 + FiniteFieldExtension(0 + 0*alpha, prime=5, alpha^2=2)x + FiniteFieldExtension(1 + 0*alpha, prime=5, alpha^2=2) over F_5)
Point P: (FiniteField(2, 5), FiniteField(3, 5))
Point Q: (FiniteFieldExtension(2 + 3*alpha, prime=5, alpha^2=2), FiniteFieldExtension(3 + 0*alpha, prime=5, alpha^2=2))
1th order: FiniteFieldExtension(1 + 0*alpha, prime=5, alpha^2=2)
----------------
2th order: FiniteFieldExtension(1 + 0*alpha, prime=5, alpha^2=2)
3th order: FiniteFieldExtension(1 + 0*alpha, prime=5, alpha^2=2)
4th order: FiniteFieldExtension(1 + 0*alpha, prime=5, alpha^2=2)
5th order: FiniteFieldExtension(1 + 0*alpha, prime=5, alpha^2=2)
6th order: FiniteFieldExtension(1 + 0*alpha, prime=5, alpha^2=2)
----------------
Identity property test passed!


In [32]:
def test_properties_equal_points():
    curve_p = EllipticCurve(a=FiniteField(0, 5), b=FiniteField(1, 5), prime=5)
    curve_q = EllipticCurve(a=FiniteFieldExtension(0, 0, 5, 2), b=FiniteFieldExtension(1, 0, 5, 2), prime=5)

    points_p_of_order_6 = [p for p in curve_p.find_points_of_order_n(6) if p is not None]
    points_q_of_order_6 = [p for p in curve_q.find_points_of_order_n(6) if p is not None]
    
    if not points_p_of_order_6:
        raise ValueError("No point of order 6 found in E(F_p). Change order n")
    
    if not points_q_of_order_6:
        raise ValueError("No point of order 6 found in E(F_p^2). Change order n")
    
    # 다양한 점 테스트
    for point_on_p in points_p_of_order_6:
        extended_point_p = (point_on_p[0].to_extension(2), point_on_p[1].to_extension(2))
        weil = WeilPairing(curve_q, extended_point_p, extended_point_p)
        assert weil.compute_pairing(6) == FiniteFieldExtension(1, 0, 5, 2), f"Failed for point {point_on_p}"
    
    for point_on_q in points_q_of_order_6:
        weil = WeilPairing(curve_q, point_on_q, point_on_q)
        assert weil.compute_pairing(6) == FiniteFieldExtension(1, 0, 5, 2), f"Failed for point {point_on_q}"

In [33]:
test_properties_equal_points()


In [34]:
# p = 5, n = 3 또는 n=6 일떄 서로 다른 P를 쓸경우 (P1, P2, Q)가 선형 독립인 조합 찾기 어려움
# 따라서 P1 = P2로 설정
# 구현 실수로 Weil pairing 인스턴스 생성할떄 P와 Q가 고정돼서 R 값이 계속 바뀜...
# 결국 biliner 테스트 불가능해짐...
def test_properties_bilinear1():
    curve_p = EllipticCurve(a=FiniteField(0, 5), b=FiniteField(1, 5), prime=5)
    curve_q = EllipticCurve(a=FiniteFieldExtension(0, 0, 5, 2), b=FiniteFieldExtension(1, 0, 5, 2), prime=5)

    n = 6  # n=6 대신 3으로 변경 (n divides p + 1)
    points_p_of_order_n = [p for p in curve_p.find_points_of_order_n(n) if p is not None]
    points_q_of_order_n = [q for q in curve_q.find_points_of_order_n(n) if q is not None]
    
    print(f"Points in E(F_5)[{n}]: {points_p_of_order_n}")
    print(f"Points in E(F_5^2)[{n}]: {points_q_of_order_n}")
    
    if not points_p_of_order_n:
        raise ValueError(f"No point of order {n} found in E(F_p). Change order n")
    if not points_q_of_order_n:
        raise ValueError(f"No point of order {n} found in E(F_p^2). Change order n")
    
    point_P1 = random.choice(points_p_of_order_n)
    extended_point_P1 = (point_P1[0].to_extension(2), point_P1[1].to_extension(2))
    
    point_P2 = random.choice(points_p_of_order_n)
    extended_point_P2 = (point_P2[0].to_extension(2), point_P2[1].to_extension(2))
    
    extended_added_point_P = curve_q.add_points(extended_point_P1, extended_point_P2)
    point_Q = random.choice(points_q_of_order_n)
    
    if not (curve_q.is_linear_independent(extended_added_point_P, point_Q) and
            curve_q.is_linear_independent(extended_point_P1, point_Q) and
            curve_q.is_linear_independent(extended_point_P2, point_Q)):
        print("Points are linearly dependent, choosing new Q...")
        for q in points_q_of_order_n:
            if (curve_q.is_linear_independent(extended_added_point_P, q) and
                curve_q.is_linear_independent(extended_point_P1, q) and
                curve_q.is_linear_independent(extended_point_P2, q)):
                point_Q = q
                break
        else:
            raise ValueError("Could not find linearly independent Q. Please run the test again to select new points.")
    
    print(f"P1 = {extended_point_P1}, P2 = {extended_point_P2}, P1+P2 = {extended_added_point_P}, Q = {point_Q}")
    # 동일한 분리자 사용
    
    weil_pairing_lhs = WeilPairing(curve_q, extended_added_point_P, point_Q)
    weil_pairing_rhs1 = WeilPairing(curve_q, extended_point_P1, point_Q)
    weil_pairing_rhs2 = WeilPairing(curve_q, extended_point_P2, point_Q)
    
    lhs = weil_pairing_lhs.compute_pairing(n)
    rhs1 = weil_pairing_rhs1.compute_pairing(n)
    rhs2 = weil_pairing_rhs2.compute_pairing(n)
    rhs = rhs1 * rhs2
    print(f"e(P1, Q) = {rhs1}, e(P2, Q) = {rhs2}")
    print(f"lhs = e(P1+P2, Q) = {lhs}, rhs = e(P1, Q) * e(P2, Q) = {rhs}, lhs == rhs = {lhs == rhs}")
    assert lhs == rhs, f"Failed for bilinear test lhs={lhs}, rhs={rhs}"

In [35]:
test_properties_bilinear1()

Points in E(F_5)[6]: [(FiniteField(2, 5), FiniteField(2, 5)), (FiniteField(2, 5), FiniteField(3, 5))]
Points in E(F_5^2)[6]: [(FiniteFieldExtension(2 + 0*alpha, prime=5, alpha^2=2), FiniteFieldExtension(2 + 0*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(2 + 0*alpha, prime=5, alpha^2=2), FiniteFieldExtension(3 + 0*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(3 + 0*alpha, prime=5, alpha^2=2), FiniteFieldExtension(0 + 2*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(3 + 0*alpha, prime=5, alpha^2=2), FiniteFieldExtension(0 + 3*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(4 + 1*alpha, prime=5, alpha^2=2), FiniteFieldExtension(2 + 0*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(4 + 1*alpha, prime=5, alpha^2=2), FiniteFieldExtension(3 + 0*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(4 + 4*alpha, prime=5, alpha^2=2), FiniteFieldExtension(2 + 0*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(4 + 4*alpha, prime=5, alpha^2=2), FiniteFieldExtension(3 + 0*alpha, 

ValueError: Could not find linearly independent Q. Please run the test again to select new points.

In [52]:
# p = 5, n = 3 또는 n=6 일떄 서로 다른 P를 쓸경우 (P1, P2, Q)가 선형 독립인 조합 찾기 어려움
# 따라서 P2 = O로 설정
def test_properties_bilinear2():
    curve_p = EllipticCurve(a=FiniteField(0, 5), b=FiniteField(1, 5), prime=5)
    curve_q = EllipticCurve(a=FiniteFieldExtension(0, 0, 5, 2), b=FiniteFieldExtension(1, 0, 5, 2), prime=5)

    n = 6
    points_p_of_order_6 = [p for p in curve_p.find_points_of_order_n(n) if p is not None]
    points_q_of_order_6 = [q for q in curve_q.find_points_of_order_n(n) if q is not None]
    
    if not points_p_of_order_6:
        raise ValueError("No point of order 6 found in E(F_p). Change order n")
    if not points_q_of_order_6:
        raise ValueError("No point of order 6 found in E(F_p^2). Change order n")
    
    point_P = random.choice(points_p_of_order_6)
    extended_point_P = (point_P[0].to_extension(2), point_P[1].to_extension(2))
    point_Q = random.choice(points_q_of_order_6)
    
    # 선형 독립성 확인
    if not curve_q.is_linear_independent(extended_point_P, point_Q):
        print("Points are linearly dependent, choosing new Q...")
        for q in points_q_of_order_6:
            if curve_q.is_linear_independent(extended_point_P, q):
                point_Q = q
                break
        else:
            raise ValueError("Could not find linearly independent Q")
    
    point_2P = curve_q.add_points(extended_point_P, curve_q.infinity)
    print(f"P = {extended_point_P}")
    print(f"P + O = {point_2P}")
    
    R_P = random.choice([pt for pt in curve_q.points if pt != extended_point_P and pt != curve_q.infinity])
    R_Q = random.choice([pt for pt in curve_q.points if pt != point_Q and pt != curve_q.infinity])
    
    weil_pairing_lhs = WeilPairing(curve_q, point_2P, point_Q)
    lhs = weil_pairing_lhs.compute_pairing(n)
    
    weil_pairing_rhs1 = WeilPairing(curve_q, extended_point_P, point_Q)
    weil_pairing_rhs2 = WeilPairing(curve_q, curve_q.infinity, point_Q)
    
    rhs1 = weil_pairing_rhs1.compute_pairing(n)
    rhs2 = weil_pairing_rhs2.compute_pairing(n)
    rhs = rhs1 * rhs2  
    
    res1 = lhs
    res2 = rhs1
    res3 = rhs2
    res4 = rhs
    
    for i in range(1, 6):
        res1 = res1 * lhs
        res2 = res2 * rhs1
        res3 = res3 * rhs2
        res4 = res4 * rhs
                
        
    print(f"e(P+O, Q)^6 = e(P, Q)^6 = {res1}")
    print(f"e(P, Q)^6 = {res2}")
    print(f"e(O, Q)^6 = {res3}")
    print(f"lhs = e(P+O, Q)^6 = {res1}, rhs = e(P, Q)^6 * e(O, Q)^6 = {res4}, lhs == rhs = {res1 == res4}")
    assert res1 == res4, f"Failed for bilinear test lhs={res1}, rhs={res4}"
    print("Bilinear test passed!")

In [53]:
test_properties_bilinear2()

P = (FiniteFieldExtension(2 + 0*alpha, prime=5, alpha^2=2), FiniteFieldExtension(3 + 0*alpha, prime=5, alpha^2=2))
P + O = (FiniteFieldExtension(2 + 0*alpha, prime=5, alpha^2=2), FiniteFieldExtension(3 + 0*alpha, prime=5, alpha^2=2))
e(P+O, Q)^6 = e(P, Q)^6 = FiniteFieldExtension(1 + 0*alpha, prime=5, alpha^2=2)
e(P, Q)^6 = FiniteFieldExtension(1 + 0*alpha, prime=5, alpha^2=2)
e(O, Q)^6 = FiniteFieldExtension(1 + 0*alpha, prime=5, alpha^2=2)
lhs = e(P+O, Q)^6 = FiniteFieldExtension(1 + 0*alpha, prime=5, alpha^2=2), rhs = e(P, Q)^6 * e(O, Q)^6 = FiniteFieldExtension(1 + 0*alpha, prime=5, alpha^2=2), lhs == rhs = True
Bilinear test passed!


In [36]:
def test_properties_non_degenerated():
    curve_p = EllipticCurve(a=FiniteField(0, 5), b=FiniteField(1, 5), prime=5)
    curve_q = EllipticCurve(a=FiniteFieldExtension(0, 0, 5, 2), b=FiniteFieldExtension(1, 0, 5, 2), prime=5)
    
    n = 6 
    points_p_of_order_n = [curve_q.infinity]
    points_q_of_order_n = [q for q in curve_q.find_points_of_order_n(n) if q is not None]
    
    print(f"Points in E(F_5)[{n}]: {points_p_of_order_n}")
    print(f"Points in E(F_5^2)[{n}]: {points_q_of_order_n}")
    
    if not points_p_of_order_n:
        raise ValueError(f"No point of order {n} found in E(F_p). Change order n")
    if not points_q_of_order_n:
        raise ValueError(f"No point of order {n} found in E(F_p^2). Change order n")
    
    point_P1 = random.choice(points_p_of_order_n)
    point_Q = random.choice(points_q_of_order_n)
    
    for point_Q in points_q_of_order_n:    
        if not (curve_q.is_linear_independent(point_P1, point_Q) and
            curve_q.is_linear_independent(point_P1, point_Q) and
            curve_q.is_linear_independent(extended_point_P2, point_Q)):
            print("Points are linearly dependent, choosing new Q...")
            for q in points_q_of_order_n:
                if (curve_q.is_linear_independent(point_P1, q) and
                    curve_q.is_linear_independent(point_P1, q) and
                    curve_q.is_linear_independent(extended_point_P2, q)):
                    point_Q = q
                    break
        else:
            raise ValueError("Could not find linearly independent Q. Please run the test again to select new points.")
        
        weil_pairing = WeilPairing(curve_q, point_P1, point_Q)
        result = weil_pairing.compute_pairing(n)
        assert result != curve_q.infinity  , f"Failed for non-generated test result={result}"
        
    print("Pass non-degenerated test")

In [37]:
test_properties_non_degenerated()

Points in E(F_5)[6]: [None]
Points in E(F_5^2)[6]: [(FiniteFieldExtension(0 + 2*alpha, prime=5, alpha^2=2), FiniteFieldExtension(2 + 4*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(0 + 2*alpha, prime=5, alpha^2=2), FiniteFieldExtension(3 + 1*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(0 + 3*alpha, prime=5, alpha^2=2), FiniteFieldExtension(2 + 1*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(0 + 3*alpha, prime=5, alpha^2=2), FiniteFieldExtension(3 + 4*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(2 + 0*alpha, prime=5, alpha^2=2), FiniteFieldExtension(2 + 0*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(2 + 0*alpha, prime=5, alpha^2=2), FiniteFieldExtension(3 + 0*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(2 + 2*alpha, prime=5, alpha^2=2), FiniteFieldExtension(0 + 1*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(2 + 2*alpha, prime=5, alpha^2=2), FiniteFieldExtension(0 + 4*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(2 + 3*alpha, prime=5, alpha^2=

In [38]:
import random

def test_properties_nth_root():
    curve_p = EllipticCurve(a=FiniteField(0, 5), b=FiniteField(1, 5), prime=5)
    curve_q = EllipticCurve(a=FiniteFieldExtension(0, 0, 5, 2), b=FiniteFieldExtension(1, 0, 5, 2), prime=5)

    n = 6
    points_p_of_order_6 = [p for p in curve_p.find_points_of_order_n(6) if p is not None]
    points_q_of_order_6 = [p for p in curve_q.find_points_of_order_n(6) if p is not None]
    
    if not points_p_of_order_6:
        raise ValueError("No point of order 6 found in E(F_p). Change order n")
    if not points_q_of_order_6:
        raise ValueError("No point of order 6 found in E(F_p^2). Change order n")
    
    point_P = random.choice(points_p_of_order_6)
    extended_point_P = (point_P[0].to_extension(2), point_P[1].to_extension(2))
    point_Q = random.choice(points_q_of_order_6)
    
    # 선형 독립성 확인
    if not curve_q.is_linear_independent(point_P, point_Q):
        print("Points are linearly dependent, choosing new Q...")
        for q in points_q_of_order_6:
            if curve_q.is_linear_independent(point_P, q):
                point_Q = q
                break
        else:
            raise ValueError("Could not find linearly independent Q")

    weil_pairing = WeilPairing(curve_q, extended_point_P, point_Q)
    result = weil_pairing.compute_pairing(6)
    print(f'{1}th order: {result}')
    power_6 = result
    print("----------------")
    for i in range(1, 6):
        power_6 = power_6 * result
        print(f'{i+1}th order: {power_6}')        
    print("----------------")    
    assert power_6 == FiniteFieldExtension(1, 0, 5, 2), f"Identity property failed: e(P, Q)^6 = {power_6} != 1"
    print("Identity property test passed!")

In [ ]:
test_properties_nth_root()

Points are linearly dependent, choosing new Q...
1th order: FiniteFieldExtension(4 + 0*alpha, prime=5, alpha^2=2)
----------------
2th order: FiniteFieldExtension(1 + 0*alpha, prime=5, alpha^2=2)
3th order: FiniteFieldExtension(4 + 0*alpha, prime=5, alpha^2=2)
4th order: FiniteFieldExtension(1 + 0*alpha, prime=5, alpha^2=2)
5th order: FiniteFieldExtension(4 + 0*alpha, prime=5, alpha^2=2)
6th order: FiniteFieldExtension(1 + 0*alpha, prime=5, alpha^2=2)
----------------
Identity property test passed!
